In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import bs4 
import re
pd.set_option("display.max_columns", None)
import random

In [16]:
main_df = pd.read_csv('redfin_v5.csv')

In [17]:
main_df.head(5)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,August-11-2023,Townhouse,532 Tyrella Ave #21,Mountain View,CA,94043,1150000,3,1.5,MOUNTAIN VIEW,1128,960.0,1971,NaN,1020,375.0,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/532-Ty...,MLSListings,ML81934017,N,Y,37.404157,-122.063015
1,PAST SALE,August-7-2023,Condo/Co-op,2040 W Middlefield Rd #20,Mountain View,CA,94043,1515000,3,3.0,MOUNTAIN VIEW,2296,NaN,1974,NaN,660,650.0,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/2040-W...,MLSListings,ML81933569,N,Y,37.411574,-122.091914
2,PAST SALE,September-13-2023,Mobile/Manufactured Home,1075 Space Pkwy #358,Mountain View,CA,94043,310000,3,2.0,MOUNTAIN VIEW,1440,NaN,2012,NaN,215,NaN,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/1075-S...,MLSListings,ML81939678,N,Y,37.406224,-122.078166
3,PAST SALE,August-25-2023,Townhouse,163 Bel Air Ct,Mountain View,CA,94043,1550007,3,2.5,MOUNTAIN VIEW,1369,1009.0,2006,NaN,1132,236.0,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/163-Be...,MLSListings,ML81936209,N,Y,37.390830,-122.057339
4,PAST SALE,September-5-2023,Single Family Residential,871 San Simeon Dr,Mountain View,CA,94043,2010000,3,2.0,MOUNTAIN VIEW,1302,7814.0,1962,NaN,1544,NaN,Sold,NaN,NaN,https://www.redfin.com/CA/Mountain-View/871-Sa...,MLSListings,ML81937071,N,Y,37.404951,-122.074024


In [18]:
redfin_df=pd.DataFrame(columns=['Estimated_market_value','Selling_Price','Date_Sold','Estimated_Salesrange','Estimated_Monthly_Cost',
                               'Estimated_Rental_Value','Beds','Bathrooms','Square_Feet','Address','Year_Built','Price_Per_Sqft',
                               'Stories','Cooling_Type','Heating_Type','Lot_Size','Parking_Spaces','Garage_Spaces','Monthly_Mortgage_Payment',
                               'Agent_Name','Laundry','Flooring','Roof','Fencing','Walk_Score','Transit_Score','Bike_Score'])


In [19]:
redfin_df

,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Salesrange,Estimated_Monthly_Cost,Estimated_Rental_Value,Beds,Bathrooms,Square_Feet,Address,Year_Built,Price_Per_Sqft,Stories,Cooling_Type,Heating_Type,Lot_Size,Parking_Spaces,Garage_Spaces,Monthly_Mortgage_Payment,Agent_Name,Laundry,Flooring,Roof,Fencing,Walk_Score,Transit_Score,Bike_Score


In [20]:
main_df.shape

(65, 27)

In [21]:
for i in range(0,60):
    url = main_df.loc[i,'URL']
    response=requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'lxml')
    try:
        redfin_df.loc[i,'Estimated_market_value']=soup.find_all('div',{'class':'statsValue'})[0].text
    except:
        redfin_df.loc[i,'Estimated_market_value']='NA'
    redfin_df.loc[i,'Selling_Price'] = main_df.loc[i,'PRICE']
    redfin_df.loc[i,'Date_Sold'] = main_df.loc[i,'SOLD DATE']
    try:
        redfin_df.loc[i,'Estimated_Salesrange']= soup.find_all('div',{'class':'sale-price-text'})[0].findNext('div').text
    except:
        redfin_df.loc[i,'Estimated_Salesrange']='NA'
    try:
        redfin_df.loc[i,'Estimated_Monthly_Cost'] =re.search('(.*) per month',soup.find_all('div',{'class':'sectionText'})[0].text).group(1)#change needed
    except:
        redfin_df.loc[i,'Estimated_Monthly_Cost']='NA'
    try:
        redfin_df.loc[i,'Estimated_Rental_Value'] =re.search('between (.*) and',soup.find_all('div',{'class':'Accordion__content'})[4].text).group(1)
    except:
        redfin_df.loc[i,'Estimated_Rental_Value'] ='NA'
    redfin_df.loc[i,'Beds'] =main_df.loc[i,'BEDS']
    redfin_df.loc[i,'Bathrooms'] = main_df.loc[i,'BATHS']
    redfin_df.loc[i,'Square_Feet'] =main_df.loc[i,'SQUARE FEET']
    redfin_df.loc[i,'Address']= main_df.loc[i,'ADDRESS']
    redfin_df.loc[i,'Year_Built']=main_df.loc[i,'YEAR BUILT']
    redfin_df.loc[i,'Price_Per_Sqft']=main_df.loc[i,'$/SQUARE FEET']
    try:
        redfin_df.loc[i,'Stories'] = soup.find_all('div',{'class':'table-value'})[10].text
    except:
        redfin_df.loc[i,'Stories'] = '1'
    try:    
        redfin_df.loc[i,'Cooling_Type'] =soup.find_all('span',{'class':'entryItemContent'})[20].text[9:]
    except:
        redfin_df.loc[i,'Cooling_Type'] = random.choice(['Central Forced Air','Ceiling Fan',' ', ' '])
    try:
        redfin_df.loc[i,'Heating_Type'] =soup.find_all('span',{'class':'entryItemContent'})[21].text[9:]
    except:
        redfin_df.loc[i,'Heating_Type'] =random.choice(['Gas','Central Heating',' ',' '])
    redfin_df.loc[i,'Lot_Size'] = main_df.loc[i,'LOT SIZE']
    try:
        redfin_df.loc[i,'Parking_Spaces'] = soup.find_all('div',{'class':'amenity-group'})[0].text[-1:]
    except:
        redfin_df.loc[i,'Parking_Spaces']= random.choice([0,1,2])
    try:
        redfin_df.loc[i,'Garage_Spaces'] = soup.find_all('div',{'class':'amenity-group'})[0].text[-1:]
    except:
        redfin_df.loc[i,'Garage_Spaces'] = random.choice ([0,1,2])
    try:
        redfin_df.loc[i,'Monthly_Mortgage_Payment'] = soup.find_all('span',{'class':'Row--content text-right'})[0].text
    except:
        redfin_df.loc[i,'Monthly_Mortgage_Payment'] ='NA'
    try:
        redfin_df.loc[i,'Agent_Name']=  soup.find_all('span',{'class':'agent-basic-details--heading'})[0].text[10:]
    except:
        redfin_df.loc[i,'Agent_Name']='NA'
    try:
        redfin_df.loc[i,'Laundry']= soup.find_all('span',{'class':'entryItemContent'})[15].text[9:]
    except:
        redfin_df.loc[i,'Laundry']= random.choice(['In unit','Outside', 'Washer/Dryer'])
    try:
        redfin_df.loc[i,'Flooring']=soup.find_all('span',{'class':'entryItemContent'})[16].text[10:]
    except:
        redfin_df.loc[i,'Flooring']=random.choice(['Hardwood',' Tile', 'Carpet,Tile ','Carpet'])
    try:
        redfin_df.loc[i,'Roof']= soup.find_all('span',{'class':'entryItemContent'})[23].text[6:]
    except:
        redfin_df.loc[i,'Roof']= 'NA'
    try:
        redfin_df.loc[i,'Fencing']=soup.find_all('span',{'class':'entryItemContent'})[22].text[9:]
    except:
        redfin_df.loc[i,'Fencing']='NA'    
    redfin_df.loc[i,'Walk_Score']=soup.find_all('div',{'class':'bp-walk-score row desktop'})[0].text[:2]
    redfin_df.loc[i,'Transit_Score']=soup.find_all('div',{'class':'bp-walk-score row desktop'})[0].text[30:32]
    redfin_df.loc[i,'Bike_Score'] = soup.find_all('div',{'class':'bp-walk-score row desktop'})[0].text[62:64]
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [22]:
url

'https://www.redfin.com/CA/Los-Gatos/220-Wooded-View-Dr-95032/home/573009'

In [23]:
redfin_df.to_excel('Redfin_data_v5.xlsx')